In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import lightgbm as lgb
import math
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression
import warnings
import pdb
warnings.filterwarnings('ignore')
import random
import torch
import torch.nn as nn
import random
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
# from torch.utils.data import TensorDataset,SequentialSampler,RandomSampler,DataLoader
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)  # 禁止hash随机化
    torch.manual_seed(seed)


set_seed(42)
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

print('训练集的大小为： ', len(train))
print('测试集的大小为： ', len(test))


# train = train.dropna()
# train.reset_index(drop=True,inplace=True)

label = 'Main_steam_flow_rate'

# =============================================================================
# ['时间', '主蒸汽流量', 'CO含量', 'HCL含量', 'NOx含量', 'SO2含量', '一次风调门', '一次风量',
#        '主蒸汽流量设定值', '二次风调门', '二次风量', '引风机转速', '推料器启停', '推料器手动指令',
#        '推料器自动投退信号', '推料器自动指令', '氧量设定值', '汽包水位', '炉排启停', '炉排实际运行指令',
#        '炉排手动指令', '炉排自动投退信号', '给水流量']
# =============================================================================


In [ ]:
train.columns = ['Time', 'Main_steam_flow_rate', 'CO_content', 'HCL_content', 'NOx_content', 'SO2_content', 'Primary_air_regulator', 'Primary_air_volume',
       'Main_steam_flow_rate_setting', 'Secondary_air_regulator', 'Secondary_air_volume', 'Inducer_fan_speed', 'Pusher_start_stop', 'Pusher_manual_command',
       'Pusher_automatic_throwback_signal', 'Pusher_automatic_command', 'Oxygen_setpoint', 'Ladle_water_level', 'Grate_start_stop', 'Grate_actual_operation_command',
       'Grate_manual_command', 'Grate_automatic_throw-out_signal', 'Feedwater_flow']

test.columns = ['Time', 'CO_content', 'HCL_content', 'NOx_content', 'SO2_content', 'Primary_air_regulator', 'Primary_air_volume',
       'Main_steam_flow_rate_setting', 'Secondary_air_regulator', 'Secondary_air_volume', 'Inducer_fan_speed', 'Pusher_start_stop', 'Pusher_manual_command',
       'Pusher_automatic_throwback_signal', 'Pusher_automatic_command', 'Oxygen_setpoint', 'Ladle_water_level', 'Grate_start_stop', 'Grate_actual_operation_command',
       'Grate_manual_command', 'Grate_automatic_throw-out_signal', 'Feedwater_flow']


In [ ]:
len(train.columns), len(test.columns)

In [ ]:
df = pd.concat([train, test])

df.info()

In [ ]:
print('df.shape: ', df.shape)

df.drop(columns=['Time'], inplace=True)


In [ ]:
df.head(5)

In [ ]:
df['gas'] = df.CO_content + df.HCL_content + df.SO2_content + df.NOx_content

# features_names = ['CO_content', 'HCL_content', 'NOx_content', 'SO2_content', 'Primary_air_regulator', 'Primary_air_volume',
#        'Main_steam_flow_rate_setting', 'Secondary_air_regulator', 'Secondary_air_volume', 'Inducer_fan_speed', 'Pusher_manual_command',
#        'Pusher_automatic_command', 'Oxygen_setpoint', 'Ladle_water_level', 'Grate_actual_operation_command',
#        'Grate_manual_command', 'Feedwater_flow']
features_names = ['Feedwater_flow', 'Oxygen_setpoint', 'Primary_air_volume', 'Main_steam_flow_rate_setting', 'Ladle_water_level']
for f in tqdm(features_names):

    # 滑动窗口特征
    window_list = [300, 600, 900]
    for win_size in window_list:
        df[f+'_fore_steps_mean_'+str(win_size)] = df[f].rolling(window=win_size, center=False).mean()
        df[f+'_fore_steps_max_'+str(win_size)] = df[f].rolling(window=win_size, center=False).max()
        df[f+'_fore_steps_min_'+str(win_size)] = df[f].rolling(window=win_size, center=False).min()
        df[f+'_fore_steps_std_'+str(win_size)] = df[f].rolling(window=win_size, center=False).std()
        df[f+'_fore_steps_skew_'+str(win_size)] = df[f].rolling(window=win_size, center=False).skew()

    # n阶差分特征
    diff_list = list(range(1, 60))
    for diff_size in diff_list:
        df[f+'_diff_'+str(diff_size)] = df[f].diff(periods=diff_size)

In [ ]:
for col in tqdm(df.columns):
    df[col].fillna(df[col].mean(), inplace=True)

In [ ]:
df['Grate_start_stop'] = df['Grate_start_stop'].map(int)
df['Pusher_start_stop'] = df['Pusher_start_stop'].map(int)
df['Pusher_automatic_throwback_signal'] = df['Pusher_automatic_throwback_signal'].map(int)
df['Grate_automatic_throw-out_signal'] = df['Grate_automatic_throw-out_signal'].map(int)



In [ ]:

# 3、内存压缩
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in tqdm(df.columns):
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (
                start_mem - end_mem) / start_mem))
    return df


# 压缩使用内存
# 由于数据比较大，所以合理的压缩内存节省空间尤为的重要
# 使用reduce_mem_usage函数可以压缩近70%的内存占有。
df = reduce_mem_usage(df)

In [ ]:
print('--------------------data describe: -----------------------------------')
df.info()
df_test = df.tail(1800)
df_train = df.iloc[:-1800, :]
df_train.reset_index(drop=True,inplace=True)

df_valid = df_train.iloc[-1800:, :]
df_train = df_train.iloc[:-1800, :]

df_valid.reset_index(drop=True, inplace=True)

In [ ]:
print('df_train.shape: ', df_train.shape)
print('df_valid.shape: ', df_valid.shape)
print('df_test.shape: ', df_test.shape)

In [ ]:
df_scale = df.iloc[:-1800, :]

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler = scaler.fit(df)
df_for_training_scaled = scaler.transform(df_train)
df_for_validing_scaled = scaler.transform(df_valid)
# df_for_testing_scaled=scaler.transform(df_test)

In [ ]:
df_for_training_scaled

In [ ]:
def createXY(dataset,n_past):
  dataX = []
  dataY = []
  for i in range(n_past, len(dataset)):
      dataX.append(dataset[i - n_past:i, 0:dataset.shape[1]])
      dataY.append(dataset[i,0])
  return np.array(dataX),np.array(dataY)


SEQ_LEN = 30 # 即时间窗口


trainX, trainY=createXY(df_for_training_scaled,SEQ_LEN)
validX, validY=createXY(df_for_validing_scaled,SEQ_LEN)
# testX, testY=createXY(df_for_testing_scaled,SEQ_LEN)

In [ ]:
print('构建时间序列特征')
print('trainX.shape, trainY.shape: ', trainX.shape, trainY.shape)
print('validX.shape, validY.shape: ', validX.shape, validY.shape)
# print('testX.shape, testY.shape: ', testX.shape, testY.shape)

In [ ]:
class My_Dataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        sample = dict(input=torch.FloatTensor(self.data[idx]),
                      label=torch.FloatTensor(np.array(self.labels[idx])))

        return sample


In [ ]:
def create_data_loader(data=None, labels=None, batch_size=32):
    ds=My_Dataset(
        data = data,
        labels = labels)
    return DataLoader(ds, batch_size=batch_size, shuffle=False)
    # if test_mode:
    #     return DataLoader(ds, batch_size=batch_size, shuffle=False)
    # else:
    #     return DataLoader(ds, batch_size=batch_size, shuffle=True)

In [ ]:
train_data_loader = create_data_loader(data=trainX,
                                       labels=trainY)

val_data_loader = create_data_loader(data=validX,
                                       labels=validY)

In [ ]:
len(train_data_loader)

In [ ]:
data = next(iter(train_data_loader))
data.keys()
print(data['input'].shape)
print(data['label'].shape)

In [ ]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('use device: ', device)

In [ ]:
# class Time_Model(nn.Module):
#
#     def __init__(self):
#         super().__init__()
#
#         self.hidden_size = 256
#         self.num_layers = 1
#         self.bidirectional = True
#         self.lstm = nn.LSTM(input_size=22,
#                              hidden_size=self.hidden_size,
#                              num_layers=self.num_layers,
#                              bidirectional=self.bidirectional,
#                              batch_first=True)
#
#         self.fc = nn.Linear(self.hidden_size*2, 1)
#
#
#     def forward(self, x):
#         device = x.device
#         batch_size, seq_len, emb_dim = x.shape
#         # 初始化：双向就乘2
#         h0 = torch.randn(self.num_layers * 2, batch_size, self.hidden_size).to(device)
#         c0 = torch.randn(self.num_layers * 2, batch_size, self.hidden_size).to(device)
#
#         x, (_, _) = self.lstm(x, (h0, c0))
#         x = x[:,-1,:] # 取最后一个时间步
#         x = self.fc(x)
#
#         return x


class Time_Model(nn.Module):

    def __init__(self):
        super().__init__()

        self.hidden_size = 256
        self.input_size = 393
        self.num_layers = 1
        self.bidirectional = True
        # self.lstm = nn.LSTM(input_size=22,
        #                      hidden_size=self.hidden_size,
        #                      num_layers=self.num_layers,
        #                      bidirectional=self.bidirectional,
        #                      batch_first=True)

        self.seq_emb = nn.Sequential(
            nn.Linear(self.input_size, self.input_size),
            nn.LayerNorm(self.input_size),
            nn.GELU(),
            nn.Dropout(0.1),
        )

        self.lstm1 = nn.LSTM(self.input_size, self.hidden_size//2, dropout=0.1, batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(self.hidden_size//2 * 2, self.hidden_size//4, dropout=0.1, batch_first=True, bidirectional=True)
        self.lstm3 = nn.LSTM(self.hidden_size//4 * 2, self.hidden_size//8, dropout=0.1, batch_first=True, bidirectional=True)


        self.head = nn.Sequential(
            # nn.Linear(self.hidden_size//8 * 2, self.hidden_size//8 * 2),
            nn.LayerNorm(self.hidden_size//8 * 2),
            nn.GELU(),
            #nn.Dropout(0.),
            nn.Linear(self.hidden_size//8 * 2, 1))


    def forward(self, x):
        device = x.device
        batch_size, seq_len, emb_dim = x.shape
        x = self.seq_emb(x)


        # 初始化：双向就乘2
        h0 = torch.randn(self.num_layers * 2, batch_size, self.hidden_size//2).to(device)
        c0 = torch.randn(self.num_layers * 2, batch_size, self.hidden_size//2).to(device)
        x, (_, _) = self.lstm1(x, (h0, c0))

         # 初始化：双向就乘2
        h0 = torch.randn(self.num_layers * 2, batch_size, self.hidden_size//4).to(device)
        c0 = torch.randn(self.num_layers * 2, batch_size, self.hidden_size//4).to(device)
        x, (_, _) = self.lstm2(x, (h0, c0))

        # 初始化：双向就乘2
        h0 = torch.randn(self.num_layers * 2, batch_size, self.hidden_size//8).to(device)
        c0 = torch.randn(self.num_layers * 2, batch_size, self.hidden_size//8).to(device)
        x, (_, _) = self.lstm3(x, (h0, c0))


        x = x[:,-1,:] # 取最后一个时间步
        x = self.head(x)

        return x


# self.seq_emb = nn.Sequential(
#             nn.Linear(len(cfg.cont_seq_cols), self.hidden_size),
#             nn.LayerNorm(self.hidden_size),
#             nn.GELU(),
#             #nn.Dropout(0.1),
#         )
#         self.lstm1 = nn.LSTM(self.hidden_size, self.hidden_size//2, dropout=0.1, batch_first=True, bidirectional=True)
#         self.lstm2 = nn.LSTM(self.hidden_size//2 * 2, self.hidden_size//4, dropout=0.1, batch_first=True, bidirectional=True)
#         self.lstm3 = nn.LSTM(self.hidden_size//4 * 2, self.hidden_size//8, dropout=0.1, batch_first=True, bidirectional=True)
#         self.head = nn.Sequential(
#             # nn.Linear(self.hidden_size//8 * 2, self.hidden_size//8 * 2),
#             nn.LayerNorm(self.hidden_size//8 * 2),
#             nn.GELU(),
#             #nn.Dropout(0.),
#             nn.Linear(self.hidden_size//8 * 2, 1),

# class Time_Model(nn.Module):
#
#     def __init__(self, input_dim=22, lstm_dim=256,
#                      dense_dim=256, logit_dim=256, num_classes=1):
#         super().__init__()
#
#         self.mlp = nn.Sequential(
#         nn.Linear(input_dim, dense_dim // 2),
#         nn.ReLU(),
#         nn.Linear(dense_dim // 2, dense_dim),
#         nn.ReLU())
#
#         self.lstm = nn.LSTM(dense_dim, lstm_dim, batch_first=True, bidirectional=True)
#
#         self.logits = nn.Sequential(
#             nn.Linear(lstm_dim * 2, logit_dim),
#             nn.ReLU(),
#             nn.Linear(logit_dim, num_classes),
#         )
#
#     def forward(self, x):
#         features = self.mlp(x)
#         features, _ = self.lstm(features)
#         features = features[:,-1,:]
#         pred = self.logits(features)
#
#         return pred

In [ ]:
model = Time_Model()
model = model.to(device)

In [ ]:
# MSE开方就是RMSE了，所以损失可以直接用来作为评估指标哦
loss_fn = torch.nn.MSELoss(reduction='mean').to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
EPOCHS = 32

In [ ]:
def train_epoch(model, data_loader, loss_fn, optimizer, device):
    model = model.train()
    losses = []

    pred_list = []
    target_list = []

    for inputs in tqdm(data_loader):
        targets = inputs["label"].to(device)
        x = inputs['input'].to(device)
        outputs = model(x)
        preds = outputs


        pred_list.extend(preds.cpu().detach().numpy().tolist())
        target_list.extend(targets.cpu().detach().numpy().tolist())


        loss = loss_fn(outputs, targets)

        losses.append(loss.item())
        loss.backward()



        # nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        optimizer.zero_grad()
        # scheduler.step()

    mean_loss = np.mean(losses)


    return mean_loss

In [ ]:
def eval_model(model, data_loader, loss_fn, device):
    model = model.eval() # 验证预测模式
    losses = []
    pred_list = []
    target_list = []

    with torch.no_grad():
        for inputs in tqdm(data_loader):
            targets = inputs["label"].to(device)
            x = inputs['input'].to(device)
            outputs = model(x)
            preds = outputs


            pred_list.extend(preds.cpu().detach().numpy().tolist())
            target_list.extend(targets.cpu().detach().numpy().tolist())


            loss = loss_fn(outputs, targets)

            losses.append(loss.item())



    mean_loss = np.mean(losses)

    return mean_loss

In [ ]:

from collections import defaultdict
history = defaultdict(list) # 记录10轮loss和acc
best_mse = float('inf')


# -------------------控制早停--------------
early_stop_epochs = 2
no_improve_epochs = 0


for epoch in range(EPOCHS):

    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_mse = train_epoch(
        model,
        train_data_loader,
        loss_fn,
        optimizer,
        device,
    )

    print(f'train_mse : {train_mse } \n ')

    val_mse = eval_model(
        model,
        val_data_loader,
        loss_fn,
        device
    )

    print(f'val_mse : {val_mse } \n ')
    print()

    history['train_mse '].append(train_mse)
    history['val_mse '].append(val_mse)


    if val_mse  < best_mse :
        print('best model saved!!!!!!!!!!!!!')
        torch.save(model.state_dict(), f'./save model/best_model.pt')
        best_mse  = val_mse

        no_improve_epochs = 0

    else:
        no_improve_epochs += 1



    if no_improve_epochs == early_stop_epochs:
        print('no improve score !!! stop train !!!')
        break

In [ ]:
df_test


In [ ]:
# 改变时间窗口后，这里要改动
df_days_past = df_valid[-SEQ_LEN:]
df_days_past.reset_index(drop=True,inplace=True)
df_days_past.info()
df_days_past.head(5)

In [ ]:
df_days_future = df_test
df_days_future.info()
df_days_future['Main_steam_flow_rate'] = 0

df_days_future.head(5)

In [ ]:
# print('df_train.shape: ', df_train.shape)
# print('df_valid.shape: ', df_valid.shape)
# print('df_test.shape: ', df_test.shape)

In [ ]:
old_scaled_array=scaler.transform(df_days_past)
new_scaled_array=scaler.transform(df_days_future)
new_scaled_df=pd.DataFrame(new_scaled_array)
print('new_scaled_array.shape: ', new_scaled_array.shape)
new_scaled_df.iloc[:,0]=np.nan
full_df=pd.concat([pd.DataFrame(old_scaled_array),new_scaled_df]).reset_index().drop(["index"],axis=1)
full_df.info()

In [ ]:
full_df

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('use device: ', device)
model = Time_Model()


path = './save model/best_model.pt'
model.load_state_dict(torch.load(path))
model = model.to(device)


model.eval()

In [ ]:
full_df_scaled_array=full_df.values
all_data=[]
time_step=SEQ_LEN
for i in range(time_step,len(full_df_scaled_array)):
    data_x=[]
    data_x.append(
    full_df_scaled_array[i-time_step :i , 0:full_df_scaled_array.shape[1]])
    data_x=np.array(data_x)

    data_x = torch.FloatTensor(data_x).to(device)
    with torch.no_grad():
        # print('-'*10)
        # print(data_x[0][0])
        prediction=model(data_x)
        # print(prediction)
        # print('-'*10)
    # print(prediction.shape)
    # break
    prediction = prediction.squeeze(-1)
    prediction = prediction.cpu().detach().numpy()

    all_data.append(prediction)
    full_df_scaled_array[i, 0] = prediction
    # full_df.iloc[i,0]=prediction

In [ ]:
full_df_scaled_array[:, 0][-10:]

In [ ]:
# full_df_scaled_array[:, 0][-10:]

In [ ]:
# all_data

In [ ]:
full_df.info()

In [ ]:
new_array=np.array(all_data)
new_array=new_array.reshape(-1,1)
prediction_copies_array = np.repeat(new_array,393, axis=-1)
y_pred_future_days = scaler.inverse_transform(np.reshape(prediction_copies_array,(len(new_array),393)))[:,0]
print(y_pred_future_days)

In [ ]:

len(all_data)

In [ ]:
prediction_copies_array


In [ ]:
test = pd.read_csv('data/test.csv')

In [ ]:

sub = pd.DataFrame({'ID': list(range(1,1801)),
                    'Time': test['时间'],
                    'Steam_flow': y_pred_future_days})


sub.to_csv('result.csv',index=False)

In [ ]:

sub = pd.read_csv('./demo2.csv')

In [ ]:
import matplotlib.pyplot as plt
train = pd.read_csv('data/train.csv')
t1 = train['主蒸汽流量'].tolist()
t2 = sub['Steam_flow'].tolist()
plt.figure(dpi=200)
plt.plot(list(range(len(t1))), t1)
plt.plot(list(range(len(t1), len(t1)+len(t2))), t2)
plt.savefig('./pred.jpg')